In [34]:
#https://developers.google.com/earth-engine/api_docs#eefeatureget
import ee
import datetime
import IPython.display
import bqplot
import ipywidgets
import pprint
pp = pprint.PrettyPrinter(indent=2) # for printing pretty idk what it is... print with pp.pprint(print stuff)
import pandas as pd
import traitlets
import dateutil.parser
import numpy as np
import ipyleaflet
from ipyleaflet import (LayersControl, basemaps, basemap_to_tiles, LayerGroup)

def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)


# Initialize
ee.Initialize()

#To-Do


In [84]:
# Dates
startTime = datetime.datetime(2013,1,1)
endTime = datetime.datetime(2019,5,1)

# AOI
ried = ee.Collection.loadTable('users/tillmueller1990/ried_roi')
germany = ee.Collection.loadTable('USDOS/LSIB_SIMPLE/2017').filterMetadata('country_co','equals','GM')
germany_1 = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na','Germany'))

# Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte();

# Paint all the polygon edges with the same number and width, display.
germany_outline = empty.paint(germany, 1, 3)


## Precipitaion Data (Radolan DWD)

In [19]:
#Load Radolan from Assest 
radolan = ee.ImageCollection('users/tillmueller1990/radolan/r_13_19')


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## Soil Moisture

In [105]:
#Soil Moisture Datasets

#NASA-USDA Global Soil Moisture Data / 3 days / 2010- 2019 / 0.25 arc degrees / SMOS lvl2 integrated into two-layer palmer model
SMOS = ee.ImageCollection('NASA_USDA/HSL/soil_moisture')
surface_moisture = SMOS.select('ssm').first().clip(clc_vector) #units mm, min=0 max=25
subsurface_moisture = SMOS.select('susm') #units: mm min=0, max=275
moisture_profile = SMOS.select('smp') #units: fraction, min0, max 1

#NASA-USDA SMAP Global Soil Moisture Data / 3 days / 2015 - 2019 / 0.25 arc degrees / SMAP level 3 + two-layer Palmer
SMAP = ee.ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture')

#GLDAS-2.1: Global Land Data Assimilation System / 3 hours / 0.25 arc degrees / 2000 - 2019 / 

#Datasets
su_m_data = surface_moisture.toArray().getInfo()
pp.pprint(su_m_data)

{ 'bands': [ { 'crs': 'EPSG:4326',
               'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0],
               'data_type': { 'dimensions': 1,
                              'precision': 'float',
                              'type': 'PixelType'},
               'dimensions': [1440, 600],
               'id': 'array'}],
  'type': 'Image'}


In [ ]:
#Derived from Sentinel 1 A/B 2014-10 - Present
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')


## Corine Land Cover 

In [102]:
# Load last corine land cover image 
clc = ee.Image.load('COPERNICUS/CORINE/V18_5_1/100m/2012', -1)
# Mask areas where soil moisture measurements valid (farmland)
clc_mask = clc.gte(11)and(clc.lte(17)) #binary map for updateMask
clc = clc.updateMask(clc_mask) #set mask for not Farmland
clc = clc.clip(germany) #clip to AOI 
clc_vector = clc.reduceToVectors(geometry=germany_1, crs=clc.projection(), scale=350, geometryType='polygon', eightConnected = False)


## Viewer

In [104]:
#Visualization Parameters
dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
germany_viz = GetTileLayerUrl(germany_outline.visualize())
#radolan_viz = GetTileLayerUrl(radolan.visualize(min=0,max=40,bands=['b1']))
clc_viz = GetTileLayerUrl(clc.visualize(min=0,max=7,palette=['FFE6FF','FFFFA8','FFFF00','E6E600','E68000','F2A64D','E6A600']))
surface_moisture_viz = GetTileLayerUrl(surface_moisture.visualize(min=0,max=28,palette=['0300ff', '418504', 'efff07', 'efff07', 'ff0303']))

# Create layer group
layer_group = LayerGroup(layers=(dark_matter_layer,ipyleaflet.TileLayer(url=clc_viz,name='Corine Land Cover 2012'), ipyleaflet.TileLayer(url=germany_viz, name='german boundary'), ipyleaflet.TileLayer(url=surface_moisture_viz, name='GSMD surface moisture')))

#Map options
center,zoom = [12.727661, 51.229822],4

#Interactive Visualizations
map1 = ipyleaflet.Map(center=center, zoom=zoom, layout={'height' : '400px'})
dc = ipyleaflet.DrawControl()
map1.add_control(dc)
map1.add_layer(layer_group)
map1.add_control(LayersControl())
map1


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [31]:
#Get Coordinates from Drawn Geometry
dc.last_draw

{'type': 'Feature',
 'properties': {'style': {'shapeOptions': {},
   'stroke': True,
   'color': '#3388ff',
   'weight': 4,
   'opacity': 0.5,
   'fill': True,
   'fillColor': None,
   'fillOpacity': 0.2,
   'clickable': True,
   'zIndexOffset': 2000,
   'icon': {'options': {}, '_initHooksCalled': True},
   'repeatMode': False}},
 'geometry': {'type': 'Point', 'coordinates': [12.727661, 51.229822]}}

In [ ]:
#Command Collection
.size().getInfo() # Get collection size
.bandNames().getInfo() # Get List with alls Bands from ee.Image not ImageCollection
.geometry().bounds().getInfo() # get Geometry of a Feature // Get bounding box of this geometry

#Vectorizing
var vectors = zones.addBands(nl2012).reduceToVectors({
  geometry: FeatureCollection,
  crs: image.projection(),
  scale: 1000,
  geometryType: 'polygon',
  eightConnected: false,
  labelProperty: 'zone',
  reducer: ee.Reducer.mean()
});

#Masking
.clip(feature)
var image = ee.Image
var mask = image.gte(2).and(lt(5))
var maskedImage = image.updateMask(mask)

#Visualizations
thumbnail_url = image.getThumbUrl({
    'bands' : '',
    'min' : ,
    'max' : ,
    'region' : .geometry().bounds().getInfo() #must be a geojson 
})
IPython.display.HTML('Thumnail URL: <a href={0}>{0}</a>'.format(thumbnail_url)) #create url to view
IPython.display.Image(url=thumbnail_url) # view direct in notebook

#Interactive Visualizations
import ipyleaflet
map1 = ipyleaflet.Map(zoom=3, layout={'height' : '400px'})
dc = ipyleaflet.DrawControl()
map1.add_control(dc)
map1
dc.last_draw # gives information about the last drawn polygon (coordinates etc.)

#Function to create a tile layer urlfrom an gee image object
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

#style the image
tile_url = GetTileLayerUrl(image.visualize(min=0, max=3000, gamma=1.5, bands=['','','']))
map1.add_layer(ipyleaflet.TileLayer(url=tile_url))
#or create layer groups 